<a href="https://colab.research.google.com/github/LuanK63/6351071043_LAB1/blob/main/6351071043_LAB1_Bai6(trang_45).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

6. (Lập trình) Cho mẫu dữ liệu Groceries Dataset2 về các giao dịch trong một tháng của
một cửa hàng outlet, gồm 9835 giao dịch và 169 items.
Với min_sup từ 0.01% đến 0.1%, hãy:


a) Tìm tất cả các tập phổ biến trong từng trường hợp bằng giải thuật Apriori và ghi
nhận thời gian xử lý của thuật toán. Gợi ý: sinh viên có thể sử dụng module
OneHotTransactions hoặc TransactionEncoder của thư viện mlxtend để chuyển dữ
liệu về dạng one-hot encoding trước khi đưa vào thuật toán.


In [2]:
!pip install mlxtend

import pandas as pd
import matplotlib.pyplot as plt
import warnings
import time
import csv
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpgrowth
warnings.filterwarnings("ignore")

In [3]:
file_path = 'groceries.csv'
dataset = []
with open(file_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        if row: dataset.append(row)

In [ ]:
print("Nguyễn Thành Luân - 6351071043")

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset, sparse=True)

df = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)

print(f"-> Dữ liệu đã sẵn sàng. Kích thước: {df.shape}")
print(f"-> Kiểu dữ liệu: {df.dtypes[0]}")

print("\n" + "="*30)
print("BẮT ĐẦU CÂU A: APRIORI")
print("="*30)

support_range = [0.0001, 0.0005, 0.001]
times_apriori = []

for min_sup in support_range:
    print(f"\n-> Đang chạy min_sup = {min_sup} ({min_sup*100:.2f}%)...")

    start_time = time.time()

    try:

        results_apriori = apriori(df,
                                  min_support=min_sup,
                                  use_colnames=True,
                                  low_memory=True)

        end_time = time.time()
        exec_time = end_time - start_time
        times_apriori.append(exec_time)

        print(f"   Xong! Thời gian: {exec_time:.4f} giây")
        print(f"   Tìm thấy {len(results_apriori)} tập phổ biến.")

    except MemoryError:
        print(f"   !!! LỖI TRÀN RAM (Out of Memory) tại mức {min_sup}.")
        print("   Apriori sinh ra quá nhiều ứng viên vượt quá giới hạn Colab.")
        times_apriori.append(None)

print("\nDanh sách thời gian chạy (giây):", times_apriori)

Nguyễn Thành Luân - 6351071043
-> Dữ liệu đã sẵn sàng. Kích thước: (9835, 169)
-> Kiểu dữ liệu: Sparse[bool, 0]

BẮT ĐẦU CÂU A: APRIORI

-> Đang chạy min_sup = 0.0001 (0.01%)...


b) Tìm tất cả các tập phổ biến trong từng trường hợp bằng giải thuật FP-Growth và ghi
nhận thời gian xử lý của thuật toán.


In [ ]:
print("Nguyễn Thành Luân - 6351071043")
print("=== BẮT ĐẦU CÂU B: FP-GROWTH ===")

times_fpgrowth = []

for min_sup in support_range:
    print(f"-> Đang chạy FP-Growth với min_sup = {min_sup} ({min_sup*100:.2f}%) ...")

    start_time = time.time()

    results_fpgrowth = fpgrowth(df, min_support=min_sup, use_colnames=True)

    end_time = time.time()
    exec_time = end_time - start_time
    times_fpgrowth.append(exec_time)

    print(f"   Xong! Thời gian: {exec_time:.4f} giây")
    print(f"   Tìm thấy {len(results_fpgrowth)} tập phổ biến.")

c) Vẽ biểu đồ so sánh thời gian xử lý của hai thuật toán trên với nhau. Sinh viên đưa ra
nhận xét và giải thích.

In [ ]:
print("Nguyễn Thành Luân - 6351071043")
print("=== BẮT ĐẦU CÂU C: SO SÁNH ===")

result_df = pd.DataFrame({
    'Min Support': labels,
    'Apriori (giây)': times_apriori,
    'FP-Growth (giây)': times_fpgrowth
})

print("Bảng so sánh thời gian xử lý:")
print(result_df)

plt.figure(figsize=(10, 6))

plt.plot(result_df['Min Support'], result_df['Apriori (giây)'],
         marker='o', color='red', linewidth=2, label='Apriori')

plt.plot(result_df['Min Support'], result_df['FP-Growth (giây)'],
         marker='s', color='blue', linewidth=2, label='FP-Growth')

plt.title('So sánh thời gian: Apriori vs FP-Growth')
plt.xlabel('Min Support (Độ hỗ trợ tối thiểu)')
plt.ylabel('Thời gian thực thi (Giây)')
plt.legend()
plt.grid(True, linestyle='--')

plt.gca().invert_xaxis()

plt.show()

print("\n--- NHẬN XÉT ---")
print("1. Tốc độ: FP-Growth chạy nhanh hơn hẳn so với Apriori ở mọi ngưỡng support.")
print("2. Xu hướng: Khi min_sup càng giảm (về phía 0.01%), thời gian chạy của Apriori tăng vọt (dốc đứng),")
print("   trong khi FP-Growth vẫn giữ được tốc độ ổn định hơn.")
print("3. Lý do: Apriori tốn nhiều chi phí quét CSDL nhiều lần để sinh ứng viên.")
print("   FP-Growth chỉ quét CSDL 2 lần và dùng cấu trúc cây FP-Tree nén dữ liệu nên hiệu quả hơn nhiều.")